# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f62583a04c0>
├── 'a' --> tensor([[ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336]])
└── 'x' --> <FastTreeValue 0x7f62583a06a0>
    └── 'c' --> tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                        [-0.1499, -0.1265,  0.7953, -1.0303],
                        [-1.0464, -0.5841,  0.0530,  1.9354]])

In [4]:
t.a

tensor([[ 0.3255,  0.9593, -0.2690],
        [-1.0058,  0.6567,  0.1336]])

In [5]:
%timeit t.a

62.4 ns ± 0.0507 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f62583a04c0>
├── 'a' --> tensor([[-0.0868,  0.8232, -0.8263],
│                   [-0.2500,  0.2756,  1.4317]])
└── 'x' --> <FastTreeValue 0x7f62583a06a0>
    └── 'c' --> tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                        [-0.1499, -0.1265,  0.7953, -1.0303],
                        [-1.0464, -0.5841,  0.0530,  1.9354]])

In [7]:
%timeit t.a = new_value

71 ns ± 0.029 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3255,  0.9593, -0.2690],
               [-1.0058,  0.6567,  0.1336]]),
    x: Batch(
           c: tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3255,  0.9593, -0.2690],
        [-1.0058,  0.6567,  0.1336]])

In [11]:
%timeit b.a

55.7 ns ± 0.0715 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7705,  0.8923,  0.1426],
               [ 0.5218,  0.2810, -0.1168]]),
    x: Batch(
           c: tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.111 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

873 ns ± 0.163 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f61b3cf5460>
├── 'a' --> tensor([[[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]],
│           
│                   [[ 0.3255,  0.9593, -0.2690],
│                    [-1.0058,  0.6567,  0.1336]]])
└── 'x' --> <FastTreeValue 0x7f61b7cfc6d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f61b3cdbac0>
├── 'a' --> tensor([[ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336],
│                   [ 0.3255,  0.9593, -0.2690],
│                   [-1.0058,  0.6567,  0.1336]])
└── 'x' --> <FastTreeValue 0x7f61ad0c0f70>
    └── 'c' --> tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                        [-0.1499, -0.1265,  0.7953, -1.0303],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.7 µs ± 547 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3571,  0.2290, -0.3192,  0.4842],
                       [-0.1499, -0.1265,  0.7953, -1.0303],
                       [-1.0464, -0.5841,  0.0530,  1.9354]],
              
                      [[ 0.3571,  0.2290, -0.3192,  0.4842],
                       [-0.1499, -0.1265,  0.7953, -1.0303],
                       [-1.0464, -0.5841,  0.0530,  1.9354]],
              
                      [[ 0.3571,  0.2290, -0.3192,  0.4842],
                       [-0.1499, -0.1265,  0.7953, -1.0303],
                       [-1.0464, -0.5841,  0.0530,  1.9354]],
              
                      [[ 0.3571,  0.2290, -0.3192,  0.4842],
                       [-0.1499, -0.1265,  0.7953, -1.0303],
                       [-1.0464, -0.5841,  0.0530,  1.9354]],
              
                      [[ 0.3571,  0.2290, -0.3192,  0.4842],
                       [-0.1499, -0.1265,  0.7953, -1.0303],
                       [-1.0464, -0.5841,  0.0530,  1.9354]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354],
                      [ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354],
                      [ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354],
                      [ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354],
                      [ 0.3571,  0.2290, -0.3192,  0.4842],
                      [-0.1499, -0.1265,  0.7953, -1.0303],
                      [-1.0464, -0.5841,  0.0530,  1.9354],
                      [ 0.3571,  0.2290, -0.3192,  0.4842],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
